In [45]:
import pymssql
import pandas as pd
import pyodbc
conn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                     "Server=Martin-PC\SQLEXPRESS;"
                     "Database=FootballData;"
                     "Trusted_Connection=yes;")

#conn = pymssql.connect(server="localhost", user="",password="", port=63642)  # You can lookup the port number inside SQL server. 
stmt = "SELECT * FROM Matches"
# Excute Query here
df_matches = pd.read_sql(stmt,conn)
print(df_matches.shape)
df_matches.head(3)

(5070, 10)


,Id,ExternalId,Date,Country,League,Season,Stage,Player_Id,AwayTeam_Id,HomeTeam_Id
0,1,1723985,2014-08-18 15:00:00,England,Premier League,2014/2015,1,None,16,15
1,2,1723982,2014-08-18 15:00:00,England,Premier League,2014/2015,1,None,12,9
2,3,1723983,2014-08-18 15:00:00,England,Premier League,2014/2015,1,None,14,11


In [161]:
stmt = "SELECT * FROM Corners"
# Excute Query here
df_corners = pd.read_sql(stmt,conn)
#df_corners = df_corners['Minute'].notnull()
print(df_corners.shape)
df_corners.head()

(30029, 6)


,Id,ExternalId,MatchId,Minute,Player1,TeamId
0,1,3652595,1723983,5,None,8455
1,2,3647429,1723982,15,None,9825
2,3,3649357,1723985,9,None,8650
3,4,3645873,1723984,16,None,8197
4,5,3647542,1723982,30,None,9825


In [305]:
stmt = "SELECT * FROM Goals"
# Excute Query here
df_goals = pd.read_sql(stmt,conn)
print(df_goals.shape)
df_goals.head()

(8299, 8)


,Id,MatchId,ExternalId,Minute,ScorerId,AssistId,Type,TeamId
0,1,1723983,3652629,14,37240,24372,shot,8191
1,2,1723985,3649433,23,246575,156008,shot,8650
2,3,1723984,3645927,20,32448,None,shot,8668
3,4,1723982,3647567,35,26777,35345,header,9826
4,5,1723984,3645943,22,26554,None,loose_ball,8197


In [382]:
result = pd.merge(df_corners.set_index("MatchId"),df_matches.set_index("ExternalId"), how='left')
#df.set_index("id"), df2.set_index("user_id"), left_index=True, right_index=True,  how="outer"

#result_1718 = result[result.Season == '2017/2018']

#premier_league = result[(result['Season']  == '2017/2018') & (result['League']  == 'Premier League')]

premier_league = result[ (result['Season'] == '2017/2018') & (result['League'] == 'Ligue 1')]
print(result.shape)

#groups = premier_league.groupby(['MatchId'])
#groups.groups.keys()
#df_goals.head()
#groups = df_corners.groupby(['MatchId'])
#groups.groups.keys()
df_goals.head()
minutes_sum = df_corners.groupby('MatchId')['Minute','TeamId'].apply(
    lambda x: "{%s}" % ', '.join(x.Minute + ' ' + x.TeamId))
#minutes_sum = df_corners.groupby('MatchId')['MatchId', 'TeamId','Minute']
#print(premier_league[:10].head())
#print(result_premier_league_1718)
premier_league_minutes = premier_league[premier_league['Minute'].notnull()]
print(premier_league_minutes.shape)
minutes_sum.head(10)
#print(list(premier_league_minutes.columns.values))

(30029, 13)
(373, 13)


MatchId
1709697    {15 9847, 44 9847, 51 9847, 52 9837, 73 9847, ...
1709698        {20 8592, 20 8592, 36 7794, 43 8592, 52 8592}
1709699    {19 7819, 21 4087, 30 7819, 54 7819, 58 7819, ...
1709700    {20 9853, 23 9853, 24 9747, 37 9853, 45 9747, ...
1709702    {6 9748, 18 9748, 45 9748, 48 9748, 60 9851, 2...
1709703    {19 9829, 20 9829, 38 8689, 42 8689, 43 9829, ...
1709704    {13 9827, 30 10249, 45 10249, 45 10249, 80 102...
1709705        {20 9830, 38 8588, 46 8588, 49 9830, 57 9830}
1709707    {1 9827, 2 9827, 7 9829, 10 9827, 42 9829, 43 ...
1709708    {4 8639, 15 8639, 16 8639, 18 8639, 18 8639, 2...
dtype: object

In [364]:
goals = pd.merge(df_goals.set_index("MatchId"), df_matches.set_index("ExternalId"), how='left')
result = goals[goals['Minute'].notnull()]
print(result.shape)
premier_league = goals[(goals['League'] == 'Ligue 1') & (goals['Season'] == '2017/2018')]
premier_league.head()
print(premier_league.shape)

(8299, 15)
(373, 15)


In [392]:
groups = df_goals.groupby(['MatchId'])
groups.groups.keys()
df_goals.head()
minutes_sum = df_goals.groupby('MatchId')['MatchId', 'TeamId','Minute']
minutes_sum.head()
#print(minutes_sum.shape)
#df_goals.head(10)
#groups.groups['1709877']

,MatchId,TeamId,Minute
0,1723983,8191,14
1,1723985,8650,23
2,1723984,8668,20
3,1723982,9826,35
4,1723984,8197,22
5,1723982,9825,45
6,1723985,8466,56
7,1723983,8455,17
8,1723984,8668,45
9,1723983,8455,21
